In [41]:
import networkx as nx
import numpy as np
from scipy.stats import norm, gaussian_kde
from scipy.integrate import quad
import pymc as pm


In [42]:
def preprocess_data(self):
        data = {}
        for var in self.unit_vars:
            for i in range(len(self.sizes)):
                data[var+str(i)] = self.data[var+str(i)]
        for var in self.subunit_vars:
            for i in range(len(self.sizes)):
                s = 0
                for j in range(self.sizes[i]):
                    s += self.data['_'+var+str(i)+'_'+str(j)]
                data[var+str(i)] = s / self.sizes[i]
        return data

In [54]:

class HierarchicalBayesSampler:
    def __init__(self, graph, data, unit_vars, subunit_vars, sizes):
        self.graph = nx.DiGraph(graph)
        self.data = data
        self.unit_vars = unit_vars
        self.subunit_vars = subunit_vars
        self.sizes = sizes
        self.processed_data = self.preprocess_data()
        
    def preprocess_data(self):
        data = {}
        for var in self.unit_vars:
            for i in range(len(self.sizes)):
                data[var+str(i)] = self.data[var+str(i)]
        for var in self.subunit_vars:
            for i in range(len(self.sizes)):
                s = 0
                for j in range(self.sizes[i]):
                    s += self.data['_'+var+str(i)+'_'+str(j)]
                data[var+str(i)] = s / self.sizes[i]
        return data

    def generate(self):  # num_samples est maintenant 2
        num_samples = self.sizes[0]
        with pm.Model() as model:
            # Hyperpriors
            mu = {var: pm.Normal(f'mu_{var}', mu=0, sigma=1) for var in self.unit_vars + self.subunit_vars}
            sigma = {var: pm.HalfNormal(f'sigma_{var}', sigma=1) for var in self.unit_vars + self.subunit_vars}

            # Priors pour les variables de niveau supérieur
            variables = {}
            for var in self.unit_vars:
                variables[var] = pm.Normal(var, mu=mu[var], sigma=sigma[var], shape=len(self.sizes))

            # Priors pour les variables de niveau inférieur (sous-unités)
            subunit_variables = {}
            for var in self.subunit_vars:
                for i in range(len(self.sizes)):
                    subunit_variables[f'{var}{i}'] = pm.Normal(f'{var}{i}', mu=mu[var], sigma=sigma[var], shape=self.sizes[i])

            # Likelihood pour les variables de niveau supérieur
            for var in self.unit_vars:
                pm.Normal(f'obs_{var}', mu=variables[var], sigma=1, 
                        observed=np.array([self.processed_data[f'{var}{i}'] for i in range(len(self.sizes))]))

            # Likelihood pour les variables de niveau inférieur (sous-unités)
            for var in self.subunit_vars:
                for i in range(len(self.sizes)):
                    pm.Normal(f'obs_{var}{i}', 
                            mu=subunit_variables[f'{var}{i}'], 
                            sigma=1, 
                            observed=np.array([self.data[f'_{var}{i}_{j}'] for j in range(self.sizes[i])]))

            # Sampling
            trace = pm.sample(num_samples, return_inferencedata=False)

        # Extraction des échantillons
        generated_data = {}
        for var in self.unit_vars:
            generated_data[var] = trace[var][0]  # Prend le premier (et seul) échantillon

        # Extraction et génération des sous-unités
        for var in self.subunit_vars:
            for i in range(len(self.sizes)):
                generated_data[f'{var}{i}'] = trace[f'{var}{i}'][0]  # Prend le premier (et seul) échantillon

        return generated_data
    
    def generate_cond(self):
        num_samples = self.sizes[0]
        with pm.Model() as model:
            # Hyperpriors pour les autres variables
            mu = {var: pm.Normal(f'mu_{var}', mu=0, sigma=1) for var in self.unit_vars + ['d']}
            sigma = {var: pm.HalfNormal(f'sigma_{var}', sigma=1) for var in self.unit_vars + ['d']}

            # Priors pour les variables de niveau supérieur
            variables = {}
            for var in self.unit_vars:
                variables[var] = pm.Normal(var, mu=mu[var], sigma=sigma[var], shape=len(self.sizes))

            # b est défini comme une variable observée suivant une loi normale standard
            b = pm.Normal('b', mu=0, sigma=1, shape=(len(self.sizes), max(self.sizes)), observed=np.random.normal(0, 1, (len(self.sizes), max(self.sizes))))

            # Priors pour les autres variables de niveau inférieur (d)
            subunit_variables = {}
            for i in range(len(self.sizes)):
                subunit_variables[f'd{i}'] = pm.Normal(f'd{i}', mu=mu['d'], sigma=sigma['d'], shape=self.sizes[i])

            # Likelihood pour les variables de niveau supérieur
            for var in self.unit_vars:
                pm.Normal(f'obs_{var}', mu=variables[var], sigma=1, 
                        observed=np.array([self.processed_data[f'{var}{i}'] for i in range(len(self.sizes))]))

            # Likelihood pour d
            for i in range(len(self.sizes)):
                pm.Normal(f'obs_d{i}', 
                        mu=subunit_variables[f'd{i}'], 
                        sigma=1, 
                        observed=np.array([self.data[f'_d{i}_{j}'] for j in range(self.sizes[i])]))

            # Sampling
            trace = pm.sample(num_samples, return_inferencedata=False)

        # Extraction des échantillons
        generated_data = {}
        for var in self.unit_vars:
            generated_data[var] = trace[var][0]

        # Génération de b (toujours à partir d'une loi normale standard)
        for i in range(len(self.sizes)):
            generated_data[f'b{i}'] = np.random.normal(0, 1, size=self.sizes[i])

        # Extraction de d
        for i in range(len(self.sizes)):
            generated_data[f'd{i}'] = trace[f'd{i}'][0]

        return generated_data




In [64]:
import numpy as np

def generate_causal_data(n_schools, n_students):
    data = {}
    
    # Generate a
    for i in range(n_schools):
        data[f'a{i}'] =np.random.normal(0, 1)
    
    # Generate b based on a
    for i in range(n_schools):
        for j in range(n_students):
            data[f'_b{i}_{j}'] = (data[f'a{i}']+1)**2 + np.random.normal(0, 1)
    
    # Generate c based on a and b
    for i in range(n_schools):
        b_mean = np.mean([data[f'_b{i}_{j}'] for j in range(n_students)])
        data[f'c{i}'] = data[f'a{i}'] +(b_mean+1)**2 + np.random.normal(0, 1)
    
    # Generate d based on b and c
    for i in range(n_schools):
        for j in range(n_students):
            data[f'_d{i}_{j}'] = (data[f'_b{i}_{j}']+1)**2 +data[f'c{i}'] + np.random.normal(0, 0.5)
    
    # Generate e based on c and d
    for i in range(n_schools):
        d_mean = np.mean([data[f'_d{i}_{j}'] for j in range(n_students)])
        data[f'e{i}'] = data[f'c{i}'] + (d_mean+1)**2 + np.random.normal(0, 1)
    
    return data

# Usage
n_schools = 10
n_students = 50
graph = [('a', '_b'), ('a', 'c'), ('_b', 'c'), ('c', '_d'), ('_b', '_d'), ('_d', 'e'), ('c', 'e')]
data = generate_causal_data(n_schools, n_students)

In [11]:
n_schools = 50
n_students = 50
# Example usage
graph = [('a', '_b'), ('a', 'c'), ('_b', 'c'), ('c', '_d'), ('_b', '_d'), ('_d', 'e'), ('c', 'e')]
data = {
    **{f'a{i}': i + 1 for i in range(n_schools)},
    **{f'c{i}': i + 5 for i in range(n_schools)},
    **{f'e{i}': i + 11 for i in range(n_schools)}
}

# Generate data for b and d
for i in range(n_schools):
    for j in range(n_students):
        data[f'_b{i}_{j}'] = 2*i + j % 2 + 1  # This creates a slight variation between students
        data[f'_d{i}_{j}'] = 2*i + j % 2 + 7  # This creates a slight variation between students

unit_vars = ['a', 'c', 'e']
subunit_vars = ['b', 'd']
sizes = [n_students] * n_schools



sampling

In [95]:

sampler = HierarchicalBayesSampler(graph, data, unit_vars, subunit_vars, sizes)


In [96]:
generated_data = sampler.generate()

Only 50 samples per chain. Reliable r-hat and ESS diagnostics require longer chains for accurate estimate.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [mu_a, mu_c, mu_e, mu_b, mu_d, sigma_a, sigma_c, sigma_e, sigma_b, sigma_d, a, c, e, b0, b1, b2, b3, b4, b5, b6, b7, b8, b9, b10, b11, b12, b13, b14, b15, b16, b17, b18, b19, b20, b21, b22, b23, b24, b25, b26, b27, b28, b29, b30, b31, b32, b33, b34, b35, b36, b37, b38, b39, b40, b41, b42, b43, b44, b45, b46, b47, b48, b49, d0, d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12, d13, d14, d15, d16, d17, d18, d19, d20, d21, d22, d23, d24, d25, d26, d27, d28, d29, d30, d31, d32, d33, d34, d35, d36, d37, d38, d39, d40, d41, d42, d43, d44, d45, d46, d47, d48, d49]


Output()

Output()

Sampling 2 chains for 1_000 tune and 50 draw iterations (2_000 + 100 draws total) took 289 seconds.
The number of samples is too small to check convergence reliably.


In [97]:
print( generated_data)


{'a': array([ 1.63065515,  1.7902874 ,  3.64379826,  4.20047022,  5.45190432,
        6.2507369 ,  7.24115763,  8.26437485, 10.2955575 , 10.88087505,
        9.52584466, 12.00288655, 14.51193756, 13.28003449, 14.05469803,
       13.87207749, 16.33657466, 19.49652471, 19.69615   , 19.35336748,
       19.77505565, 22.01298776, 23.0668964 , 24.23471286, 26.58541814,
       27.80412966, 27.22020489, 27.80914964, 29.2355082 , 29.61532339,
       29.50518518, 32.38111287, 32.66135276, 32.95779157, 35.86792962,
       36.30651974, 37.76048832, 39.49839915, 38.49655652, 39.32357128,
       38.85188431, 41.19294883, 41.18251028, 43.25409831, 45.93033013,
       45.32301705, 45.29567944, 47.19897617, 48.98364624, 50.2227248 ]), 'c': array([ 5.18359223,  6.57355743,  7.04102518,  6.76520866,  8.96573395,
        9.88228739, 11.62027158, 11.13115107, 13.83440859, 13.78365585,
       13.96381998, 16.57984348, 16.9390979 , 16.86406085, 20.5648649 ,
       19.55889189, 20.82874816, 20.15223973, 21.07

In [ ]:

generated_data_cond = sampler.generate_cond()

Only 50 samples per chain. Reliable r-hat and ESS diagnostics require longer chains for accurate estimate.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [mu_a, mu_c, mu_e, mu_d, sigma_a, sigma_c, sigma_e, sigma_d, a, c, e, d0, d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11, d12, d13, d14, d15, d16, d17, d18, d19, d20, d21, d22, d23, d24, d25, d26, d27, d28, d29, d30, d31, d32, d33, d34, d35, d36, d37, d38, d39, d40, d41, d42, d43, d44, d45, d46, d47, d48, d49]


Output()

Output()

In [ ]:
print(generated_data_cond)

In [ ]:
from scipy.stats import entropy

def kl_divergence(p, q):
    """
    Calcule la divergence KL entre deux distributions empiriques représentées par des tableaux.
    
    :param p: Premier tableau de données
    :param q: Second tableau de données
    :return: Valeur de la divergence KL
    """
    # Assurez-vous que les tableaux ont la même taille
    min_len = min(len(p), len(q))
    p = p[:min_len]
    q = q[:min_len]
    
    # Calculez les histogrammes des deux distributions
    bins = np.linspace(min(np.min(p), np.min(q)), max(np.max(p), np.max(q)), 1000)
    p_hist, _ = np.histogram(p, bins=bins, density=True)
    q_hist, _ = np.histogram(q, bins=bins, density=True)
    
    # Ajoutez un petit epsilon pour éviter la division par zéro
    epsilon = 1e-10
    p_hist += epsilon
    q_hist += epsilon
    
    # Normalisez les histogrammes
    p_hist /= np.sum(p_hist)
    q_hist /= np.sum(q_hist)
    
    # Calculez la divergence KL
    return entropy(p_hist, q_hist)


In [ ]:
# Compute KL divergence for 'e' variable
e_generated = generated_data['e']
e_original = np.array([sampler.processed_data[f'e{i}'] for i in range(len(sizes))])

e_generated_cond = generated_data_cond['e']




In [ ]:
kl_div = kl_divergence(e_original, e_generated)
print(f"KL divergence between original 'e' and generated 'e': {kl_div}")
kl_div_cond = kl_divergence(e_original, e_generated_cond)
print(f"KL divergence between original 'e' and generated 'e' with conditionning: {kl_div_cond}")


NEW MODEL

In [1]:
import pymc as pm
import numpy as np


In [2]:

# Hypothèses : Tu as des données disponibles sous forme de matrices
# A, C, E : (100,)
# B, D : (100, 50)

A = np.random.normal(0, 1, 100)
C = np.random.normal(0, 1, 100)
E = np.random.normal(0, 1, 100)

B_observed = np.random.normal(0, 1, (100, 50))
D_observed = np.random.normal(0, 1, (100, 50))


In [3]:
with pm.Model() as model:
    # Priors pour les coefficients (au niveau des écoles)
    alpha_B = pm.Normal('alpha_B', mu=0, sigma=1)
    beta_B_A = pm.Normal('beta_B_A', mu=0, sigma=1)
    beta_B_C = pm.Normal('beta_B_C', mu=0, sigma=1)
    beta_B_E = pm.Normal('beta_B_E', mu=0, sigma=1)
    
    alpha_D = pm.Normal('alpha_D', mu=0, sigma=1)
    beta_D_A = pm.Normal('beta_D_A', mu=0, sigma=1)
    beta_D_C = pm.Normal('beta_D_C', mu=0, sigma=1)
    beta_D_E = pm.Normal('beta_D_E', mu=0, sigma=1)

    # Variables aléatoires pour les écoles
    mu_B_j = alpha_B + beta_B_A * A + beta_B_C * C + beta_B_E * E
    mu_D_j = alpha_D + beta_D_A * A + beta_D_C * C + beta_D_E * E
    
    # Ajustement des dimensions pour correspondre à celles des élèves
    mu_B_j = pm.Deterministic('mu_B_j', mu_B_j[:, None])
    mu_D_j = pm.Deterministic('mu_D_j', mu_D_j[:, None])
    
    # Priors pour les écarts types au niveau des élèves
    sigma_B = pm.HalfNormal('sigma_B', sigma=1)
    sigma_D = pm.HalfNormal('sigma_D', sigma=1)
    
    # Niveau des élèves : chaque élève est modelé en fonction de son école
    B = pm.Normal('B', mu=mu_B_j, sigma=sigma_B, observed=B_observed)
    D = pm.Normal('D', mu=mu_D_j, sigma=sigma_D, observed=D_observed)

    # Inference
    trace = pm.sample(1000, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [alpha_B, beta_B_A, beta_B_C, beta_B_E, alpha_D, beta_D_A, beta_D_C, beta_D_E, sigma_B, sigma_D]


Output()

Output()

Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 12 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics


New model

In [4]:
import numpy as np

In [5]:
import pymc as pm

In [21]:
import numpy as np

def generate_causal_data(n_schools, n_students):
    data = {}
    
    # Generate a
    for i in range(n_schools):
        data[f'a{i}'] =np.random.normal(0, 1)
    
    # Generate b based on a
    for i in range(n_schools):
        for j in range(n_students):
            data[f'_b{i}_{j}'] = (data[f'a{i}']+1)**2 + np.random.normal(0, 1)
    
    # Generate c based on a and b
    for i in range(n_schools):
        b_mean = np.mean([data[f'_b{i}_{j}'] for j in range(n_students)])
        data[f'c{i}'] = data[f'a{i}'] +(b_mean+1)**2 + np.random.normal(0, 1)
    
    # Generate d based on b and c
    for i in range(n_schools):
        for j in range(n_students):
            data[f'_d{i}_{j}'] = (data[f'_b{i}_{j}']+1)**2 +data[f'c{i}'] + np.random.normal(0, 0.5)
    
    # Generate e based on c and d
    for i in range(n_schools):
        d_mean = np.mean([data[f'_d{i}_{j}'] for j in range(n_students)])
        data[f'e{i}'] = data[f'c{i}'] + (d_mean+1)**2 + np.random.normal(0, 1)
    
    return data

# Usage
n_schools = 50
n_students = 50
graph = [('a', '_b'), ('a', 'c'), ('_b', 'c'), ('c', '_d'), ('_b', '_d'), ('_d', 'e'), ('c', 'e')]
data = generate_causal_data(n_schools, n_students)

In [22]:

# Convert data to numpy arrays
a = np.array([data[f'a{i}'] for i in range(n_schools)])
c = np.array([data[f'c{i}'] for i in range(n_schools)])
e = np.array([data[f'e{i}'] for i in range(n_schools)])

# For b and d, we need to create 2D arrays
b = np.array([[data[f'_b{i}_{j}'] for j in range(n_students)] for i in range(n_schools)])
d = np.array([[data[f'_d{i}_{j}'] for j in range(n_students)] for i in range(n_schools)])

print("Shape of a:", a.shape)
print("Shape of b:", b.shape)
print("Shape of c:", c.shape)
print("Shape of d:", d.shape)
print("Shape of e:", e.shape)


Shape of a: (50,)
Shape of b: (50, 50)
Shape of c: (50,)
Shape of d: (50, 50)
Shape of e: (50,)


In [23]:
with pm.Model() as model:
    # Priors pour les coefficients (au niveau des écoles)
    alpha_B = pm.Normal('alpha_B', mu=0, sigma=1)
    beta_B_A = pm.Normal('beta_B_A', mu=0, sigma=1)
    beta_B_C = pm.Normal('beta_B_C', mu=0, sigma=1)
    beta_B_E = pm.Normal('beta_B_E', mu=0, sigma=1)
    
    alpha_D = pm.Normal('alpha_D', mu=0, sigma=1)
    beta_D_A = pm.Normal('beta_D_A', mu=0, sigma=1)
    beta_D_C = pm.Normal('beta_D_C', mu=0, sigma=1)
    beta_D_E = pm.Normal('beta_D_E', mu=0, sigma=1)

    # Variables aléatoires pour les écoles
    mu_B_j = alpha_B + beta_B_A * a + beta_B_C * c + beta_B_E * e
    mu_D_j = alpha_D + beta_D_A * a + beta_D_C * c + beta_D_E * e
    
    # Ajustement des dimensions pour correspondre à celles des élèves
    mu_B_j = pm.Deterministic('mu_B_j', mu_B_j[:, None])
    mu_D_j = pm.Deterministic('mu_D_j', mu_D_j[:, None])
    
    # Priors pour les écarts types au niveau des élèves
    sigma_B = pm.HalfNormal('sigma_B', sigma=1)
    sigma_D = pm.HalfNormal('sigma_D', sigma=1)
    
    # Niveau des élèves : chaque élève est modelé en fonction de son école
    B = pm.Normal('B', mu=mu_B_j, sigma=sigma_B, observed=b)
    D = pm.Normal('D', mu=mu_D_j, sigma=sigma_D, observed=d)

    # Inference
    trace = pm.sample(10, return_inferencedata=True, progressbar=True)
    
# Analyse des résultats
print(pm.summary(trace, var_names=['alpha_B', 'beta_B_A', 'beta_B_C', 'beta_B_E','alpha_D', 'beta_D_A', 'beta_D_C', 'beta_D_E',  'sigma_B', 'sigma_D']))

Only 10 samples per chain. Reliable r-hat and ESS diagnostics require longer chains for accurate estimate.
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (2 chains in 1 job)
NUTS: [alpha_B, beta_B_A, beta_B_C, beta_B_E, alpha_D, beta_D_A, beta_D_C, beta_D_E, sigma_B, sigma_D]


Output()

Output()

Sampling 2 chains for 1_000 tune and 10 draw iterations (2_000 + 20 draws total) took 80 seconds.
The number of samples is too small to check convergence reliably.


           mean     sd  hdi_3%  hdi_97%  mcse_mean  mcse_sd  ess_bulk  \
alpha_B   0.471  0.069   0.366    0.597      0.016    0.012      19.0   
beta_B_A  0.498  0.083   0.370    0.618      0.018    0.013      22.0   
beta_B_C  0.129  0.008   0.118    0.142      0.002    0.001      19.0   
beta_B_E -0.000  0.000  -0.000   -0.000      0.000    0.000      21.0   
alpha_D   1.632  0.194   1.409    1.967      0.038    0.027      26.0   
beta_D_A  0.267  0.266  -0.176    0.681      0.052    0.039      26.0   
beta_D_C  1.857  0.027   1.815    1.889      0.007    0.005      18.0   
beta_D_E  0.000  0.000   0.000    0.000      0.000    0.000      17.0   
sigma_B   1.021  0.012   0.998    1.036      0.002    0.002      26.0   
sigma_D   6.391  0.079   6.298    6.529      0.015    0.011      26.0   

          ess_tail  r_hat  
alpha_B       22.0   0.97  
beta_B_A      26.0   1.01  
beta_B_C      26.0   1.02  
beta_B_E      22.0   1.03  
alpha_D       26.0   1.06  
beta_D_A      26.0   1.02  


In [24]:
generated_data = {}
generated_data['a'] = a  # A est déjà une donnée observée
generated_data['c'] = c  # C est déjà une donnée observée
generated_data['e'] = e  # E est déjà une donnée observée

mu_B = trace.posterior['mu_B_j'].mean(dim=('chain', 'draw')).values
sigma_B = trace.posterior['sigma_B'].mean(dim=('chain', 'draw')).values
generated_data['b'] = np.random.normal(mu_B, sigma_B)

mu_D = trace.posterior['mu_D_j'].mean(dim=('chain', 'draw')).values
sigma_D = trace.posterior['sigma_D'].mean(dim=('chain', 'draw')).values
generated_data['d'] = np.random.normal(mu_D, sigma_D)

# Affichage des formes des données générées
for var in ['a', 'b', 'c', 'd', 'e']:
    print(f"Shape of generated {var}: {generated_data[var].shape}")
    
print("Generated data:", generated_data)


Shape of generated a: (50,)
Shape of generated b: (50, 1)
Shape of generated c: (50,)
Shape of generated d: (50, 1)
Shape of generated e: (50,)
Generated data: {'a': array([-0.0604787 , -0.51264881,  1.38441562, -1.08195618, -0.05714678,
       -0.04760641,  0.83498881,  1.92435385, -0.16650771,  0.14866284,
        1.31086853, -0.10308322,  0.12125551, -0.24738086,  0.66384591,
       -0.92465653,  0.59770634, -0.41671222, -0.43026338,  0.14300143,
       -0.64525478,  1.73629333, -0.84695515,  0.10037651, -0.01368405,
        0.95188312,  0.07152122, -1.14834939, -0.89558186, -0.29339743,
        0.06542617, -0.57566648, -0.9567297 , -0.3528315 , -0.64690748,
       -1.59648775,  0.84260452, -0.17069467, -0.63533388, -0.64902263,
        0.80177417, -0.39760034,  0.05598391,  0.78607459, -1.54639799,
        0.71411722,  1.23077759,  0.42734589, -0.44256942,  0.08569056]), 'c': array([ 3.82755371e+00,  1.33312612e+00,  4.86928947e+01,  2.05720440e-01,
        2.85468462e+00,  2.42094

In [38]:
from scipy.stats import gaussian_kde

def kl_divergence(p, q, bandwidth='scott'):
    """
    Calcule la divergence KL entre deux distributions empiriques représentées par des tableaux,
    en utilisant l'estimation de densité par noyau.
    
    :param p: Premier tableau de données
    :param q: Second tableau de données
    :param bandwidth: Méthode pour estimer la largeur de bande ('scott', 'silverman' ou un nombre)
    :return: Valeur de la divergence KL
    """
    # Assurez-vous que les tableaux ont la même taille
    min_len = min(len(p), len(q))
    p = p[:min_len]
    q = q[:min_len]
    
    # Estimation de densité par noyau
    kde_p = gaussian_kde(p, bw_method=bandwidth)
    kde_q = gaussian_kde(q, bw_method=bandwidth)
    
    # Créez un espace d'échantillonnage
    x = np.linspace(min(np.min(p), np.min(q)), max(np.max(p), np.max(q)), 10000)
    
    # Estimez les densités
    p_density = kde_p(x)
    q_density = kde_q(x)
    
    # Ajoutez un petit epsilon pour éviter la division par zéro
    epsilon = 1e-10
    p_density += epsilon
    q_density += epsilon
    
    # Normalisez les densités
    p_density /= np.sum(p_density)
    q_density /= np.sum(q_density)
    
    # Calculez la divergence KL
    return np.sum(p_density * np.log(p_density / q_density))

In [39]:
# Calcul des divergences KL
kl_divs = {}

for var in ['a', 'b', 'c', 'd', 'e']:
    original = eval(var)  # Les données originales
    generated = generated_data[var]  # Les données générées
    
    # Pour b et d, nous devons aplatir les arrays 2D
    if var in ['b', 'd']:
        original = original.flatten()
        generated = generated.flatten()
    
    kl_divs[var] = kl_divergence(original, generated)

# Affichage des résultats
for var, kl_div in kl_divs.items():
    print(f"KL divergence pour {var}: {kl_div}")

KL divergence pour a: 0.0
KL divergence pour b: 0.24859435031543628
KL divergence pour c: 0.0
KL divergence pour d: 1.1780404405825062
KL divergence pour e: 0.0
